# Copy the Pretrained Models to Torch Cache Directory

In [ ]:
import os

if not os.path.exists("/root/.cache/torch/hub/checkpoints/"):
    os.makedirs("/root/.cache/torch/hub/checkpoints/")
!cp ../input/pretrained-model-weights-pytorch/resnet18-5c106cde.pth /root/.cache/torch/hub/checkpoints/

# Imports

In [ ]:
import os
import zipfile
import shutil
import math
import time

# Preprocessing

In [ ]:
work_dir = "/kaggle/working"
dataset_dir = "/kaggle/working/data"

# Delete the residues
!rm -rf $dataset_dir

# Create the dataset directory
os.makedirs(dataset_dir)

# Change the working directory
os.chdir(work_dir)

# Unzip the data to datset_directory
zip_files = ['test1', 'train']
for zip_file in zip_files:
    with zipfile.ZipFile(f"../input/dogs-vs-cats/{zip_file}.zip", "r") as z:
        z.extractall(dataset_dir)
        print(f"{zip_file} unzipped")

# Rearrange the dataset
dataset_classes = ["cat", "dog"]
# Create the required directories
for dataset_class in dataset_classes:
    if not os.path.exists(f"{dataset_dir}/train/{dataset_class}"):
        os.makedirs(f"{dataset_dir}/train/{dataset_class}")

# Move the extracted data in the corresponding directories
files = sorted(os.listdir(f"{dataset_dir}/train"))
for file in files:
    if os.path.isfile(f"{dataset_dir}/train/{file}"):
        dataset_class = file.split('.')[0]
        shutil.move(f"{dataset_dir}/train/{file}", f"{dataset_dir}/train/{dataset_class}/{file}")

# Create the vaidation data out of training data
val_split = 0.2
for dataset_class in dataset_classes:
    if not os.path.exists(f"{dataset_dir}/val/{dataset_class}"):
        os.makedirs(f"{dataset_dir}/val/{dataset_class}")

for dataset_class in dataset_classes:
    class_train_dir = f"{dataset_dir}/train/{dataset_class}"
    class_val_dir = f"{dataset_dir}/val/{dataset_class}"
    images = os.listdir(class_train_dir)
    total_images = len(images)
    val_images = math.floor(total_images * val_split)
    slice_factor = math.floor(total_images / val_images)
    for i in range(0, total_images, slice_factor):
        shutil.move(f"{class_train_dir}/{images[i]}", f"{class_val_dir}/{images[i]}")

!echo Train Set
!ls $dataset_dir/train/cat | wc -l
!ls $dataset_dir/train/dog | wc -l

!echo Validation Set
!ls $dataset_dir/val/cat | wc -l
!ls $dataset_dir/val/dog | wc -l

# Define Dataset and Dataloader Classes

In [ ]:
import os
import random
import cv2
import numpy as np
from PIL import Image

import torch.utils.data
from torchvision import transforms

In [ ]:
class BaseDset(object):

    def __init__(self):
        self.__base_path = ""

        self.__train_set = {}
        self.__test_set = {}
        self.__train_keys = []
        self.__test_keys = []

    def load(self, base_path):
        self.__base_path = base_path
        train_dir = os.path.join(self.__base_path, 'train')
        test_dir = os.path.join(self.__base_path, 'val')

        self.__train_set = {}
        self.__test_set = {}
        self.__train_keys = []
        self.__test_keys = []

        for class_id in os.listdir(train_dir):
            class_dir = os.path.join(train_dir, class_id)
            self.__train_set[class_id] = []
            self.__train_keys.append(class_id)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.__train_set[class_id].append(img_path)

        for class_id in os.listdir(test_dir):
            class_dir = os.path.join(test_dir, class_id)
            self.__test_set[class_id] = []
            self.__test_keys.append(class_id)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.__test_set[class_id].append(img_path)

        return len(self.__train_keys), len(self.__test_keys)

    def getTriplet(self, split='train'):
        if split == 'train':
            dataset = self.__train_set
            keys = self.__train_keys
        else:
            dataset = self.__test_set
            keys = self.__test_keys

        pos_idx = 0
        neg_idx = 0
        pos_anchor_img_idx = 0
        pos_img_idx = 0
        neg_img_idx = 0

        pos_idx = random.randint(0, len(keys) - 1)
        while True:
            neg_idx = random.randint(0, len(keys) - 1)
            if pos_idx != neg_idx:
                break

        pos_anchor_img_idx = random.randint(0, len(dataset[keys[pos_idx]]) - 1)
        while True:
            pos_img_idx = random.randint(0, len(dataset[keys[pos_idx]]) - 1)
            if pos_anchor_img_idx != pos_img_idx:
                break

        neg_img_idx = random.randint(0, len(dataset[keys[neg_idx]]) - 1)

        pos_anchor_img = dataset[keys[pos_idx]][pos_anchor_img_idx]
        pos_img = dataset[keys[pos_idx]][pos_img_idx]
        neg_img = dataset[keys[neg_idx]][neg_img_idx]

        return pos_anchor_img, pos_img, neg_img

In [ ]:
RESIZE_SIZE = (192, 192)
NUM_TRAIN_SAMPLES = 10000
NUM_VAL_SAMPLES = 2500
BATCH_SIZE = 64
NUM_WORKERS = 2

means = (0.485, 0.456, 0.406)
stds = (0.229, 0.224, 0.225)
TRAIN_TRANSFORM = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                     transforms.RandomVerticalFlip(),
                                     transforms.RandomRotation(90), 
                                     transforms.ToTensor(), 
                                     transforms.Normalize(means, stds)])
TEST_TRANSFORM = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(means, stds)
                    ])

class BaseLoader(torch.utils.data.Dataset):
    def __init__(self, triplets, transform=None):
        self.triplets = triplets
        self.transform = transform

    def __getitem__(self, index):
        img1_pth, img2_pth, img3_pth = self.triplets[index]
        img1 = Image.open(img1_pth)
        img2 = Image.open(img2_pth)
        img3 = Image.open(img3_pth)

        try:
            img1 = img1.resize(RESIZE_SIZE)
        except Exception as e:
            img1 = np.zeros(RESIZE_SIZE, dtype=np.uint8)

        try:
            img2 = img2.resize(RESIZE_SIZE)
        except Exception as e:
            img2 = np.zeros(RESIZE_SIZE, dtype=np.uint8)

        try:
            img3 = img3.resize(RESIZE_SIZE)
        except Exception as e:
            img3 = np.zeros(RESIZE_SIZE, dtype=np.uint8)

        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)

        return img1, img2, img3

    def __len__(self):
        return len(self.triplets)


def get_loader():
    train_data_loader = None
    test_data_loader = None

    train_triplets = []
    test_triplets = []

    dset_obj = None
    loader = BaseLoader
    
    
    dset_obj = BaseDset()
    dset_obj.load(dataset_dir)
    for i in range(NUM_TRAIN_SAMPLES):
        pos_anchor_img, pos_img, neg_img = dset_obj.getTriplet()
        train_triplets.append([pos_anchor_img, pos_img, neg_img])
    for i in range(NUM_VAL_SAMPLES):
        pos_anchor_img, pos_img, neg_img = dset_obj.getTriplet(split='test')
        test_triplets.append([pos_anchor_img, pos_img, neg_img])

    train_data_loader = torch.utils.data.DataLoader(
        loader(train_triplets, transform=TRAIN_TRANSFORM),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_data_loader = torch.utils.data.DataLoader(
        loader(test_triplets, transform=TEST_TRANSFORM),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

    return train_data_loader, test_data_loader

# Define Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import resnet18

In [ ]:
class EmbeddingResnet2Blocks(nn.Module):
    def __init__(self, pretrained):
        super(EmbeddingResnet2Blocks, self).__init__()

        resnet = resnet18(pretrained=pretrained)
        self.features = resnet
#         self.features = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1,
#                                       resnet.layer2, resnet.avgpool)

    def forward(self, x):
        features = self.features.forward(x)
        features = features.view(features.size(0), -1)
        features = F.normalize(features, p=2, dim=1)
        return features

class TripletNet(nn.Module):
    def __init__(self, embeddingNet):
        super(TripletNet, self).__init__()
        self.embeddingNet = embeddingNet

    def forward(self, i1, i2, i3):
        E1 = self.embeddingNet(i1)
        E2 = self.embeddingNet(i2)
        E3 = self.embeddingNet(i3)
        return E1, E2, E3

def get_model(checkpoints=None, pretrained=False):
    embeddingNet = EmbeddingResnet2Blocks(pretrained=pretrained)

    model = TripletNet(embeddingNet)
    model = nn.DataParallel(model, device_ids=[0])
    model = model.to("cuda")

    # Load weights if provided
    if not checkpoints == None:
        if os.path.isfile(checkpoints):
            print("=> Loading checkpoint '{}'".format(checkpoints))
            checkpoint = torch.load(checkpoints)
            model.load_state_dict(checkpoint['state_dict'])
            print("=> Loaded checkpoint '{}'".format(checkpoints))
        else:
            print("=> No checkpoint found at '{}'".format(checkpoints))

    return model

# Train the Embeddings

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable
import torch.backends.cudnn as cudnn

In [ ]:
def save_checkpoint(state, file_name):
    torch.save(state, file_name)

def train(data, model, criterion, optimizer, epoch, file, device="cuda"):
    print("******** Training ********")
    print("******** Training ********", file=file)
    total_loss = 0
    model.train()
    for batch_idx, img_triplet in enumerate(data):
        anchor_img, pos_img, neg_img = img_triplet
        anchor_img, pos_img, neg_img = anchor_img.to(device), pos_img.to(device), neg_img.to(device)
        anchor_img, pos_img, neg_img = Variable(anchor_img), Variable(pos_img), Variable(neg_img)
        E1, E2, E3 = model(anchor_img, pos_img, neg_img)
        dist_E1_E2 = F.pairwise_distance(E1, E2, 2)
        dist_E1_E3 = F.pairwise_distance(E1, E3, 2)

        target = torch.FloatTensor(dist_E1_E2.size()).fill_(-1)
        target = target.to(device)
        target = Variable(target)
        loss = criterion(dist_E1_E2, dist_E1_E3, target)
        total_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        log_step = 10
        if (batch_idx % log_step == 0) and (batch_idx != 0):
            print('Train Epoch: {} [{}/{}] \t Loss: {:.4f}'.format(epoch, batch_idx, len(data), total_loss / log_step))
            print('Train Epoch: {} [{}/{}] \t Loss: {:.4f}'.format(epoch, batch_idx, len(data), total_loss / log_step),
                  file=file)
            total_loss = 0
    print("****************")
    print("****************", file=file)


def test(data, model, criterion, file, device="cuda"):
    metrics = {}
    print("******** Testing ********")
    print("******** Testing ********", file=file)
    with torch.no_grad():
        model.eval()
        accuracies = [0, 0, 0]
        acc_threshes = [0, 0.2, 0.5]
        total_loss = 0
        for batch_idx, img_triplet in enumerate(data):
            anchor_img, pos_img, neg_img = img_triplet
            anchor_img, pos_img, neg_img = anchor_img.to(device), pos_img.to(device), neg_img.to(device)
            anchor_img, pos_img, neg_img = Variable(anchor_img), Variable(pos_img), Variable(neg_img)
            E1, E2, E3 = model(anchor_img, pos_img, neg_img)
            dist_E1_E2 = F.pairwise_distance(E1, E2, 2)
            dist_E1_E3 = F.pairwise_distance(E1, E3, 2)

            target = torch.FloatTensor(dist_E1_E2.size()).fill_(-1)
            target = target.to(device)
            target = Variable(target)

            loss = criterion(dist_E1_E2, dist_E1_E3, target)
            total_loss += loss

            for i in range(len(accuracies)):
                prediction = (dist_E1_E3 - dist_E1_E2 - 1.0 * acc_threshes[i]).cpu().data
                prediction = prediction.view(prediction.numel())
                prediction = (prediction > 0).float()
                batch_acc = prediction.sum() * 1.0 / prediction.numel()
                accuracies[i] += batch_acc
        print('Test Loss: {}'.format(total_loss / len(data)))
        print('Test Loss: {}'.format(total_loss / len(data)), file=file)
        metrics['loss'] = total_loss / len(data)
        for i in range(len(accuracies)):
            print(
                'Test Accuracy with diff = {}% of margin: {}'.format(acc_threshes[i] * 100, accuracies[i] / len(data)))
            print('Test Accuracy with diff = {}% of margin: {}'.format(acc_threshes[i] * 100,
                                                                       accuracies[i] / len(data)), file=file)
            metrics[f"{acc_threshes[i] * 100}"] = accuracies[i] / len(data)
    print("****************")
    print("****************", file=file)

    return metrics

In [ ]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)
cudnn.benchmark = True

exp_dir = f"{work_dir}/embedding"
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

# Open the file to log the training
file = open(f"{exp_dir}/info.txt", 'w')

model = get_model(pretrained=True)
params = []
for key, value in dict(model.named_parameters()).items():
    if value.requires_grad:
        params += [{'params': [value]}]

In [ ]:
EPOCHS = 50
CKP_FREQ = 1
LEARNING_RATE = 0.0001

criterion = torch.nn.MarginRankingLoss(margin=1.0)
optimizer = optim.Adam(params, lr=LEARNING_RATE)
best_model = {}
best_metrics = None

# Train Test Loop
for epoch in range(1, EPOCHS + 1):
    # Init data loaders
    train_data_loader, test_data_loader = get_loader()
    # Test train
    if epoch == 1:
        metrics = test(test_data_loader, model, criterion, file)
    train(train_data_loader, model, criterion, optimizer, epoch, file)
    metrics = test(test_data_loader, model, criterion, file)
    # Save model
    model_to_save = {
        "epoch": epoch + 1,
        "metrics": metrics,
        'state_dict': model.state_dict(),
    }
    if best_metrics is not None:
        best_score = 0
        for i in metrics.keys():
            if i == 'loss':
                if float(metrics[i]) < float(best_metrics[i]):
                    best_score += 1
            else:
                if float(metrics[i]) > float(best_metrics[i]):
                    best_score += 1
        if best_score >= 3:
            print(f"New Best model at epoch # {epoch}")
            print(f"New Best model at epoch # {epoch}", file=file)
            best_model = model_to_save
            best_metrics = metrics
    else:
        best_metrics = metrics
    if epoch % CKP_FREQ == 0:
        file_name = os.path.join(exp_dir, "checkpoint_" + str(epoch) + ".pth")
        save_checkpoint(model_to_save, file_name)
# Save the best model
file_name = os.path.join(exp_dir, "checkpoint_best" + ".pth")
save_checkpoint(best_model, file_name)
file.close()

# Training of Classifier

## Initialize the Paths

In [ ]:
embedding_weights_path = os.path.join(exp_dir, "checkpoint_best" + ".pth")
train_dataset_path = f"{dataset_dir}/train"
val_dataset_path = f"{dataset_dir}/val"

## Imports

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import cv2
from PIL import Image

## Dataloader

In [ ]:
TRANSFORM = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(means, stds)
                    ])
AUGMENTATION = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                   transforms.RandomVerticalFlip(),
                                   transforms.RandomRotation(90)])

def image_loader_train(path):
    image = Image.open(path)
    image = image.resize(RESIZE_SIZE)
    image = AUGMENTATION(image)
    image = TRANSFORM(image)
    
    return image

def image_loader_test(path):
    image = Image.open(path)
    image = image.resize(RESIZE_SIZE)
    image = TRANSFORM(image)
    
    return image


def get_dataset_loader(dataset_root_path, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, train=True):
    if train:
        image_datset = datasets.ImageFolder(root=dataset_root_path, loader=image_loader_train)
    else:
        image_datset = datasets.ImageFolder(root=dataset_root_path, loader=image_loader_test)
    data_loader = DataLoader(dataset=image_datset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    print(f"Dataset Size: {len(image_datset)}")
    print(f"Dataset Classes: {image_datset.classes}")

    return image_datset, data_loader

## Define Model

In [ ]:
class Classifier(nn.Module):
    def __init__(self, embeddingNet, num_classes=2):
        super(Classifier, self).__init__()
        self.embeddingNet = embeddingNet
        self.fc = nn.Linear(1000, num_classes)
        self.dropout = nn.Dropout(0.4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embeddingNet(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x

def get_classfier_model(embedding_weights_path):
    embeddingNet = EmbeddingResnet2Blocks(pretrained=True)

    model = Classifier(embeddingNet)
    model = nn.DataParallel(model, device_ids=[0])
    model = model.to("cuda")

    # Load the embedding weights
    if embedding_weights_path:
        if os.path.isfile(embedding_weights_path):
            checkpoint = torch.load(embedding_weights_path)
            model.load_state_dict(checkpoint['state_dict'], strict=False)

    # Freeze the embeddingNet
    for key, value in dict(model.module.embeddingNet.named_parameters()).items():
        if value.requires_grad:
            value.requires_grad = False
    model.module.embeddingNet.train(False)

    return model

## Train the Classifier

In [ ]:
def train(data, model, criterion, optimizer, epoch, dataset_size, device="cuda"):
    print("******** Training ********")
    print("******** Training ********", file=file)
    total_loss = 0
    total_correct_predictions = 0
    model.train()
    for batch_idx, d in enumerate(data):
        inputs, labels = d
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss
        _, preds = torch.max(outputs, 1)
        total_correct_predictions += torch.sum(preds == labels.data)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        log_step = 10
        if (batch_idx % log_step == 0) and (batch_idx != 0):
            print('Train Epoch: {} [{}/{}] \t Loss: {:.4f}'.format(epoch, batch_idx, len(data), total_loss / log_step))
            print('Train Epoch: {} [{}/{}] \t Loss: {:.4f}'.format(epoch, batch_idx, len(data), total_loss / log_step),
                  file=file)
            total_loss = 0
    epoch_accuracy = float(total_correct_predictions) / dataset_size

    print(f"Accuracy: {epoch_accuracy}")
    print(f"Accuracy: {epoch_accuracy}", file=file)
    print("****************")
    print("****************", file=file)


def test(data, model, criterion, dataset_size, device="cuda"):
    metrics = {}
    print("******** Testing ********")
    print("******** Testing ********", file=file)
    with torch.no_grad():
        model.eval()
        total_loss = 0
        total_correct_predictions = 0
        for batch_idx, d in enumerate(data):
            inputs, labels = d
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss
            _, preds = torch.max(outputs, 1)
            total_correct_predictions += torch.sum(preds == labels.data)

        epoch_loss = float(total_loss) / len(data)
        epoch_accuracy = float(total_correct_predictions) / dataset_size

        metrics['loss'] = epoch_loss
        metrics['accuracy'] = epoch_accuracy

    print(f"Loss: {epoch_loss}, Accuracy: {epoch_accuracy}")
    print(f"Loss: {epoch_loss}, Accuracy: {epoch_accuracy}", file=file)
    print("****************")
    print("****************", file=file)

    return metrics

In [ ]:
EPOCHS = 20
CHK_FREQ = 1

torch.manual_seed(1)
torch.cuda.manual_seed(1)
cudnn.benchmark = True

exp_dir = f"{work_dir}/classifier"
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

# Open the file to log the training
file = open(f"{exp_dir}/info.txt", 'w')

model = get_classfier_model(embedding_weights_path)
params = []
for key, value in dict(model.named_parameters()).items():
    if value.requires_grad:
        params += [{'params': [value]}]

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(params, lr=0.001, momentum=0.9)
best_model = {}
best_metrics = None

# Train Test Loop
for epoch in range(1, EPOCHS + 1):
    # Init data loaders
    train_dataset, train_data_loader = get_dataset_loader(dataset_root_path=train_dataset_path,
                                                          batch_size=128)
    test_dataset, test_data_loader = get_dataset_loader(dataset_root_path=val_dataset_path,
                                                        batch_size=128, train=False)
    # Test train
    if epoch == 1:
        metrics = test(test_data_loader, model, criterion, len(test_dataset))
        if best_metrics is None:
            best_metrics = metrics
    train(train_data_loader, model, criterion, optimizer, epoch, len(train_dataset))
    metrics = test(test_data_loader, model, criterion, len(test_dataset))
    # Save model
    model_to_save = {
        "epoch": epoch + 1,
        "metrics": metrics,
        'state_dict': model.state_dict(),
    }
    if best_metrics is not None:
        best_score = 0
        for i in metrics.keys():
            if i == 'loss':
                if float(metrics[i]) < float(best_metrics[i]):
                    best_score += 1
            else:
                if float(metrics[i]) > float(best_metrics[i]):
                    best_score += 1
        if best_score >= 2:
            print(f"New Best model at epoch # {epoch}")
            print(f"New Best model at epoch # {epoch}", file=file)
            best_model = model_to_save
            best_metrics = metrics
    else:
        best_metrics = metrics
    if epoch % CHK_FREQ == 0:
        file_name = os.path.join(exp_dir, "checkpoint_" + str(epoch) + ".pth")
        save_checkpoint(model_to_save, file_name)
# Save the best model
file_name = os.path.join(exp_dir, "checkpoint_best" + ".pth")
save_checkpoint(best_model, file_name)
file.close()

# Inference (Preparation of CSV File)

> ## Load the Classifier Inference Model

In [ ]:
classifier_weights_path = os.path.join(exp_dir, "checkpoint_best" + ".pth")

def get_classifier_infer_model(pretrained=True):
    embeddingNet = EmbeddingResnet2Blocks(pretrained=True)
    
    model = Classifier(embeddingNet)
    model = nn.DataParallel(model, device_ids=[0])
    model = model.to("cuda")

    # Load the trained classifier weights
    if classifier_weights_path:
        if os.path.isfile(classifier_weights_path):
            checkpoint = torch.load(classifier_weights_path)
            model.load_state_dict(checkpoint['state_dict'], strict=True)

    return model

In [ ]:
infer_model = get_classifier_infer_model()
infer_model.eval()

## Perform Inference on the Test Set

In [ ]:
test_data_dir_path = f"/kaggle/working/data/test1"
MODEL_CLASSES = ['cat', 'dog']
results = {}

for i in range(1, len(os.listdir(test_data_dir_path)) + 1):
    image_path = f"{test_data_dir_path}/{i}.jpg"
    # Preprocess the image
    image = image_loader(image_path)
    image = image.to("cuda")
    image = torch.unsqueeze(image, 0)
    # Perform inference
    scores = infer_model(image)
    # Parse the inference results
    max_index = torch.argmax(scores)
    predicted_class = MODEL_CLASSES[max_index]
    results[i] = predicted_class

## Save the Results in the CSV File

In [ ]:
import csv

csv_contents = {"dog": 0, "cat": 1}
csv_file_path = "/kaggle/working/Warriors.csv"
with open(csv_file_path, 'w') as f:
    for key in results.keys():
        f.write("%s,%s\n"%(f"{key}.jpg", csv_contents[results[key]]))

* ## Delete the Datasets

In [ ]:
!rm -rf $dataset_dir